# LeNet
在线性神经网络模型中，我们将图像展成一维向量后经过全连接层处理，现在我们可以使用卷积层来处理图像数据，以此保留图像的空间结构。

在本节中，我们介绍LeNet，它是最早发布的卷积神经网络之一，因其在计算机视觉任务中的高效性能而受到广泛关注。这个模型是由AT&T贝尔实验室的研究员Yann LeCun在1989年提出的（并以其命名），目的是识别图像 中的手写数字。当时，LeNet取得了与支持向量机（support vector machines）性能相媲美的成果，成为监督学习的主流方法。
LeNet被广泛用于自动取款机（ATM）机中，帮助识别处理支票的数字。
时至今日，一些自动取款机仍在运行Yann LeCun和他的同事Leon Bottou在上世纪90年代写的代码。

## LeNet网络结构
总体来看，(**LeNet（LeNet-5）由两个部分组成：**)
* 卷积编码器：由两个卷积层组成;
* 全连接层密集块：由三个全连接层组成。

该架构如 :numref:`img_lenet`所示。
![LeNet中的数据流。输入是手写数字，输出为10种可能结果的概率。](./lenet.svg)
:label:`img_lenet`

* 每个卷积块中的基本单元是一个卷积层、一个sigmoid激活函数和平均汇聚层。请注意，虽然ReLU和最大汇聚层更有效，但它们在20世纪90年代还没有出现。
* 每个卷积层使用$5\times 5$卷积核和一个sigmoid激活函数。
* 第一卷积层有6个输出通道，而第二个卷积层有16个输出通道。
* 每个$2\times2$池操作（步幅2）通过空间下采样将维数减少4倍。
* 为了将卷积块的输出传递给稠密块，我们必须在小批量中展平每个样本。换言之，我们将这个四维输入转换成全连接层所期望的二维输入。这里的二维表示的第一个维度索引小批量中的样本，第二个维度给出每个样本的平面向量表示。
* LeNet的稠密块有三个全连接层，分别有120、84和10个输出。

下面，我们将一个大小为$28 \times 28$的单通道（黑白）图像通过LeNet。
![LeNet 的简化版。](./lenet-vert.svg)

In [47]:
import torch
import torch.nn as nn

In [48]:
class CNN_Model(torch.nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 6, kernel_size=5, padding=2),
            
            torch.nn.Sigmoid(),
            torch.nn.AvgPool2d(stride=2,kernel_size=2))
        
        self.conv2=torch.nn.Sequential(
            torch.nn.Conv2d(6, 16, kernel_size=5),
            
            torch.nn.Sigmoid(),
            torch.nn.AvgPool2d(stride=2, kernel_size=2))
        
        self.dense = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(16 * 5 * 5, 120),
            torch.nn.Sigmoid(),
            
            torch.nn.Linear(120, 84),
            torch.nn.Sigmoid(),
            torch.nn.Linear(84, 10))
        
    # 前向传播
    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(x1)
        
        x = self.dense(x2)
        return x

LeNet = CNN_Model()

## 模型训练

 现在我们已经实现了LeNet，让我们看看[**LeNet在Fashion-MNIST数据集上的表现**]。

In [49]:
import torchvision
from torch.utils import data
from torchvision import transforms

### 加载数据集

In [50]:
tran = transforms.ToTensor()
train_data = torchvision.datasets.FashionMNIST(root='../data', 
                                               train=True, 
                                               download=True, 
                                               transform=tran)
test_data = torchvision.datasets.FashionMNIST(root='../data', 
                                               train=False, 
                                               download=True, 
                                               transform=tran)

In [51]:
batch_size = 256
train_iter = data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_iter = data.DataLoader(test_data, shuffle=False, batch_size=batch_size)

### 定义损失函数和优化器

In [52]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(LeNet.parameters(), lr=0.001)

In [53]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LeNet.to(device)  

CNN_Model(
  (conv1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (dense): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=400, out_features=120, bias=True)
    (2): Sigmoid()
    (3): Linear(in_features=120, out_features=84, bias=True)
    (4): Sigmoid()
    (5): Linear(in_features=84, out_features=10, bias=True)
  )
)

### 训练

In [54]:
num_epochs = 10
for epoch in range(num_epochs):
    correct = 0
    for X, y in train_iter:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        y_hat = LeNet(X)
        l = loss(y_hat, y)  # 计算的是这个batch中所有样本的平均交叉熵损失
        l.backward()
        optimizer.step()
        # _,  pred = torch.max(y_hat.data, 1)
        # correct += torch.sum(pred == y)
        correct += (torch.argmax(y_hat, axis=1) == y).sum()
    print(f'accuracy in epoch {epoch + 1}: {correct / len(train_data)}')

accuracy in epoch 1: 0.3319000005722046
accuracy in epoch 2: 0.6690166592597961
accuracy in epoch 3: 0.7191666960716248
accuracy in epoch 4: 0.7425333261489868
accuracy in epoch 5: 0.759933352470398
accuracy in epoch 6: 0.7781333327293396
accuracy in epoch 7: 0.7932666540145874
accuracy in epoch 8: 0.805150032043457
accuracy in epoch 9: 0.816349983215332
accuracy in epoch 10: 0.8274500370025635


## 模型测试

In [56]:
with torch.no_grad():
    correct = 0 
    for X, y in test_iter:
        X, y = X.to(device), y.to(device)
        y_hat =  LeNet(X)
        correct += (torch.argmax(y_hat, axis=1) == y).sum()
    accuracy = correct / len(test_data)
    print(f'Accuracy on testing data: {accuracy}')

Accuracy on testing data: 0.8131999969482422


## 模型保存

In [63]:
torch.save(LeNet.state_dict(), 'saved_lenet')

## 模型加载

In [68]:
copy_lenet = CNN_Model().to(device)
copy_lenet.load_state_dict(torch.load('saved_lenet'))

<All keys matched successfully>

In [69]:
with torch.no_grad():
    correct = 0 
    for X, y in test_iter:
        X, y = X.to(device), y.to(device)
        y_hat =  copy_lenet(X)
        correct += (torch.argmax(y_hat, axis=1) == y).sum()
    accuracy = correct / len(test_data)
    print(f'Accuracy on testing data: {accuracy}')

Accuracy on testing data: 0.8131999969482422


加载的模型与保存的模型在测试集上的预测准确度相同，说明模型的保存和加载是正确的。